We are currently working on creating a synthetic dataset where the gains and T60s will be broadband so that we can train the simplest Diff GFDN architecture with scalar gains instead of filters. The synthetic dataset will have two rooms with two different T60s and a set of amplitudes for each source-receiver positions. While we can generate the T60 randomly for the two rooms, generating the amplitudes randomly for each position is not ideal. To generate amplitudes in a meaningful way, we want to analyse the amplitudes at 1kHz in our existing coupled room dataset.

In [ ]:
import numpy as np
import os
import torch
import pickle
import re
from pathlib import Path
import matplotlib.pyplot as plt
from numpy.typing import NDArray, ArrayLike
from typing import List, Optional, Tuple
from sklearn.mixture import GaussianMixture
from scipy.interpolate import griddata
from scipy.fft import rfft2, rfftfreq
from scipy.stats import multivariate_normal


from diff_gfdn.config.config import DiffGFDNConfig
from diff_gfdn.dataloader import RoomDataset, ThreeRoomDataset
from diff_gfdn.utils import db
from slope2noise.rooms import RoomGeometry

os.chdir('..')  # This changes the working directory to DiffGFDN

### Helper plotting functions

In [ ]:
# Function to plot ellipsoids
def plot_ellipsoid(mean : ArrayLike, cov_matrix: NDArray, ax, color='r', alpha=0.2):
    """Plot the GMM parameters as ellipsoids on a scatter plot"""
    # Decompose the covariance matrix to get the principal axes and radii
    U, s, Vt = np.linalg.svd(cov_matrix)
    radii = np.sqrt(s)
    
    # Generate data for ellipsoid surface
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = radii[0] * np.outer(np.cos(u), np.sin(v))
    y = radii[1] * np.outer(np.sin(u), np.sin(v))
    z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

    # Rotate the ellipsoid data with the eigenvectors
    for i in range(len(x)):
        for j in range(len(x)):
            [x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], U) + mean
    
    # Plot the ellipsoid
    ax.plot_surface(x, y, z, color=color, alpha=alpha)

### Read data and plot amplitudes as a function of space

In [ ]:
config_dict = DiffGFDNConfig()
room_data = ThreeRoomDataset(Path(config_dict.room_dataset_path).resolve(), config_dict)
config_dict = config_dict.model_copy(update={"num_groups": room_data.num_rooms})

common_decay_times = room_data.common_decay_times
band_centre_hz = np.array(room_data.band_centre_hz)
amplitudes = np.array(room_data.amplitudes)
num_rec_pos = room_data.num_rec

In [ ]:
# get amplitudes at 1kHz
mid_band_freq = 1000.0
idx = np.argwhere(band_centre_hz == mid_band_freq)[0][0]
amplitudes_mid_band = amplitudes[..., idx]
aperture_coords = [[(4, 3), (4, 4.5)], [(8.5, 5), (10, 5)]]
room_geom = RoomGeometry(config_dict.sample_rate, 
                         room_data.num_rooms, 
                         room_data.room_dims, 
                         room_data.room_start_coord, 
                         room_data.aperture_coords)

room_geom.plot_amps_at_receiver_points(np.array(room_data.receiver_position), np.array(room_data.source_position).squeeze(),
                                       amplitudes_mid_band.T, scatter_plot=False)

### Stats helper functions

In [ ]:
def fit_multivariate_normal_params(data: NDArray):
    """Get the mean and covariance matrix of a multivariate gaussian"""
    n, d = data.shape
    # get mean vector of size d x 1
    mean = np.mean(data, axis=0)
    covariance = ((data-mean).T @ (data-mean)) / (n-1)
    return mean, covariance

def sample_gmm(means: List, covariances : List, pis: List, component_choices: ArrayLike, n_samples:int):
    """
    Sample from a GMM with means, covariances and mixing coefficient pi
    Args:
        means (List): means of size num_mixtures x num_features
        covariances (List): covariance matrix of size num_mixtures x num_features x num_features
        pis (List): weight of each mixture (Weights must sum to 1).
        component_choise (List): which normal distribution in the mixture to sample from, of size n_samples
        n_samples (int): number of samples to return
    Returns:
        NDArray: sampled daata from GMM of shape n_samples x num_features
    """
    K = len(pis)  # Number of components
    d = means.shape[1]  # Dimensionality of data
    
    # Step 2: Sample from the chosen Gaussian component
    samples = np.zeros((n_samples, d))
    
    for i in range(n_samples):
        k = component_choices[i]  # Chosen component
        # Sample from the corresponding multivariate normal
        samples[i, :] = np.random.multivariate_normal(means[k], covariances[k])
    
    return samples


### Fit the amplitudes to a distribution

#### Fitting to Gaussian Mixture Model

If our data is multimodal, and the amplitudes are correlated, then we can model it as a linear mixture of $K$ gaussians,
\begin{aligned}
p({A_{k, \mathbf{x}}}) &= \frac{\exp \left[-(A_{k, \mathbf{x}} - \mathbf{\mu_k})^T \mathbf{\Sigma_k}^{-1} (A_{k, \mathbf{x}} - \mathbf{\mu_k}) \right]}{\sqrt{(2 \pi)^k \text{det} (\mathbf{\Sigma}_k)}} \\
p(A_{\mathbf{x}}) &= \sum_{k=1}^K \pi_k p({A_{k, \mathbf{x}}})
\end{aligned}

where $A_{k, \mathbf{x}}$ is the amplitude corresponding to the $k$th group at position $\mathbf{x}$. It is derived from a gaussian mixture model with $K$ mixtures (one corresponding to each common decay time). We wish to find the parameters $\mathbf{\Sigma_k}$ and $\mathbf{\mu_k}$ of this distribution, and then sample $A_{k,\mathbf{x}}$ from it. To do this, we will use the expectation maximisation algorithm.

#### Fitting to a single multivariate Gaussian
We can also fit a single multivariate normal distribution to the amplitudes for $K$ slopes, and calculate the mean and covariance of those. 

To check which of these is a valid choice, let's see if the data is unimodal or multimodal (should be modeled with GMM). We also fit a GMM to the data and checked the covariance matrices which had significantly large off-diagonal elements, confirming that the amplitudes in the three rooms are not independent.

**From the scatter plot, the GMM is the right choice for modeling this data. The GMM ellipsoids are shown in green and match the 3 distinct clusters. The multivariate Gaussian ellipsoid is shown in red and does not match the 3 clusters.**

In [ ]:
# mean is of shape num_groups x num_feature
# weight is of size num_groups
# covariance is of size num_groups x num_feature x num_feature
# the groups are not ordered, but we can explicitly order them. The group with the largest mean
# is likely going to belong to room 1 (as it contains the source), the second largest to room 2 and 
# third largest to room 3 (see plots above).

def sort_gmm_by_means(gmm):
    """Sort GMM components by the means along the first feature dimension."""
    
    # Sort in descending order by average value of the means
    sorted_indices = np.flip(np.argsort(np.mean(gmm.means_, axis=-1)))
    
    # Reorder the GMM parameters based on the sorted indices
    gmm.means_ = gmm.means_[sorted_indices]
    gmm.covariances_ = gmm.covariances_[sorted_indices]
    gmm.weights_ = gmm.weights_[sorted_indices]
    
    # If GMM has precisions (used in certain covariance types), reorder them as well
    if hasattr(gmm, 'precisions_'):
        gmm.precisions_ = gmm.precisions_[sorted_indices]
    if hasattr(gmm, 'precisions_cholesky_'):
        gmm.precisions_cholesky_ = gmm.precisions_cholesky_[sorted_indices]
    
    # Reorder the labels (optional, if relevant)
    gmm.predict_labels_ = sorted_indices
    return gmm


# get parameters of GMM, data of shape (N, 3)
data = amplitudes_mid_band
gmm = GaussianMixture(n_components=3)
gmm.fit(data)
sort_gmm_by_means(gmm)

# save to a pickle file
gmm_dict = {'means': gmm.means_, "covariance": gmm.covariances_, "weights": gmm.weights_}
with open(Path('resources/Georg_3room_FDTD/amplitude_distribution.pkl').resolve(), 'wb') as f:
    pickle.dump(gmm_dict, f)
print("Mean vector for all K groups:\n", db(gmm.means_))
print("Weigths for all K groups:\n", db(gmm.weights_))


# get parameters of a multivariate normal distribution
mu_n, sigma_n = fit_multivariate_normal_params(amplitudes_mid_band)
# print the covariance matrix to see if it is a diagonal
print("Covariance matrix (should be diagonal for independent variables):\n", sigma_n)


# Create a 3D scatter plot
# Plot the scatter plot
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:, 0], data[:, 1], data[:, 2], s=10, c='blue', marker='o', label='Data')

# Plot ellipsoids for each covariance matrix
for i in range(room_data.num_rooms):
    plot_ellipsoid(gmm.means_[i], gmm.covariances_[i], ax, color='green')
plot_ellipsoid(mu_n, sigma_n, ax, color='red') 

# Labels and title
ax.set_xlabel('Room 1')
ax.set_ylabel('Room 2')
ax.set_zlabel('Room 3')
plt.title('GMM Covariance Matrices and Data')
plt.show()

The question we are yet to answer is - how do we make the amplitudes a function of the room geometry and source-receiver positions? Here are some empirical rules we follow to design this function.

1. The strength of the amplitudes should be inversely proportional to the distance between the source and the receiver.
2. If the receiver is in a particular room, then the amplitudes of the slopes corresponding
   to that room should dominate.

We propose the following function,
$$
A(k, \mathbf{x}) = \frac{1}{\sqrt{d(\text{src, rec}) * d(\text{rec, room}_k) }} * \pi_k \mu_{k,\text{rec-room}}
$$
where $d(.)$ is the Euclidean distance between 2 points, $d(\text{rec, room}_k)$ is the distance between the receiver and the $k$th room, and $\mu_{k,\text{rec-room}}$ is the $k$th term of the mean vector corresponding to the room where the receiver is located.

### Geometry helper functions

In [ ]:
def generate_random_positions(N: int, L: ArrayLike, U: ArrayLike):
    """
    Generate N x 3 matrix of uniformly distributed cartesian coordinates, where the value in each column
    is constrained by its own range (L_k, U_k).

    Parameters:
    - N (int): Number of rows (samples)
    - L (list or array): Lower bounds for each column (size 3)
    - U (list or array): Upper bounds for each column (size 3)
    
    Returns:
    - np.ndarray: N x 3 array of random numbers
    """
    assert len(L) == len(U) == 3, "L and U must both have same size"
    
    # Generate N random values for each column in the range (L_k, U_k)
    x = np.random.uniform(L[0], U[0], size=N)
    y = np.random.uniform(L[1], U[1], size=N)
    z = np.random.uniform(L[2], U[2], size=N)
    
    # Stack columns together to form N x 3 array
    return np.column_stack((x, y, z))


In [ ]:
# number of receiver positions
num_samples = 10
mean_amps = np.ones((room_data.num_rooms, room_data.num_rooms))
mean_amps = gmm.weights_ * gmm.means_

# create listening points in room 1
for k in range(room_data.num_rooms):
    lower_bound = np.array(room_data.room_start_coord[k])
    upper_bound = np.array(room_data.room_start_coord[k]) + np.array(room_data.room_dims[k])
    sampled_positions = generate_random_positions(num_samples, lower_bound, upper_bound)
    amplitudes_sampled = room_geom.get_amplitude_based_on_position(sampled_positions, 
                                                                   np.squeeze(room_data.source_position),
                                                                   mean_amps)
    room_geom.plot_amps_at_receiver_points(sampled_positions,
                                           np.array(room_data.source_position).squeeze(),
                                           amplitudes_sampled,
                                           scatter_plot=False, title=f'Receivers in room {k+1}')
    # break

### Now fit GMM to amplitudes at each frequency band

In [ ]:
means = []
cov = []
weights = []
gmm = GaussianMixture(n_components=room_data.num_rooms)

for k in range(len(band_centre_hz)):
    cur_band_freq = band_centre_hz[k]
    amplitudes_cur_band = amplitudes[..., k]
    
    room_geom.plot_amps_at_receiver_points(np.array(room_data.receiver_position), np.array(room_data.source_position).squeeze(),
                                           amplitudes_cur_band.T, 
                                           scatter_plot=False,
                                           )

    # get parameters of GMM, data of shape (N, 3)
    data = amplitudes_cur_band
    gmm.fit(data)
    sort_gmm_by_means(gmm)
    means.append(gmm.means_)
    cov.append(gmm.covariances_)
    weights.append(gmm.weights_)

    # Create a 3D scatter plot
    # Plot the scatter plot
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data[:, 0], data[:, 1], data[:, 2], s=10, c='blue', marker='o', label='Data')
    
    # Plot ellipsoids for each covariance matrix
    for i in range(room_data.num_rooms):
        plot_ellipsoid(gmm.means_[i], gmm.covariances_[i], ax, color='green')
    
    # Labels and title
    ax.set_xlabel('Room 1')
    ax.set_ylabel('Room 2')
    ax.set_zlabel('Room 3')
    plt.title(f'GMM Covariance Matrices and Data for band center = {cur_band_freq:.0f}Hz')
    plt.show()
    

# save to a pickle file
gmm_dict = {"band_centre_hz": band_centre_hz, 'means': means, "covariance": cov, "weights": weights}
with open(Path('resources/Georg_3room_FDTD/amplitude_distribution_full_band.pkl').resolve(), 'wb') as f:
    pickle.dump(gmm_dict, f)

### Plot the 2D FFT of the amplitude grid to see the spatial bandwidth. 

This will give us an idea of the spatial sampling needed

#### Plot 2D amplitude grid

In [ ]:
from importlib import reload
import slope2noise
reload(slope2noise.rooms)
from slope2noise.rooms import RoomGeometry

room_geom = RoomGeometry(config_dict.sample_rate, 
                         room_data.num_rooms, 
                         room_data.room_dims, 
                         room_data.room_start_coord, 
                         room_data.aperture_coords)

#### Take 2D FFT and examine bandwidth

In [ ]:
def get_2D_fft(input_data: NDArray, grid_spacing_m: float, sound_speed: float = 343):
    num_rows, num_cols = input_data.shape
    Nfft_rows = 2**np.ceil(np.log2(num_rows)).astype('int')
    Nfft_cols = 2**np.ceil(np.log2(num_cols)).astype('int')
    spectrum = rfft2(amps_interp, (Nfft_rows, Nfft_cols))
    sound_speed = 343
    spatial_sample_rate = sound_speed / grid_spacing_m
    x_freqs = rfftfreq(Nfft_rows, d = 1.0/spatial_sample_rate)
    y_freqs= rfftfreq(Nfft_cols, d= 1.0/spatial_sample_rate)
    return spectrum, x_freqs, y_freqs
    

fig, ax = plt.subplots(room_data.num_rooms+1, 1, figsize=(6, 3 * room_data.num_rooms+1))
fig.tight_layout()

for k in range(room_data.num_rooms):
    amps_interp, hann_window = room_geom.get_2D_matrix_of_amplitudes(np.array(room_data.receiver_position), 
                                          amplitudes_mid_band[...,k], 
                                          smooth_edges=True, 
                                          plot=True, 
                                          grid_spacing_m=room_data.grid_spacing_m)
    amps_interp = np.nan_to_num(amps_interp, nan=0)
    amps_spec, x_freqs, y_freqs = get_2D_fft(amps_interp, room_data.grid_spacing_m)
    
    im = ax[k].imshow(db(amps_spec),
                  extent=(0, max(x_freqs),0, max(y_freqs)),
                  origin='lower',
                  cmap='viridis')
    fig.colorbar(im, ax=ax[k], orientation='vertical')
    fig.subplots_adjust(hspace=0.4)
    # Labels and title
    ax[k].set_xlabel('X freqs')
    ax[k].set_ylabel('Y freqs')
    ax[k].set_title(f'2D Amplitude spectrum (dB) for slope {k+1}')

    if k == room_data.num_rooms - 1:
        window_spec, _, _ = get_2D_fft(hann_window, room_data.grid_spacing_m)
        im = ax[k+1].imshow(db(window_spec),
                  extent=(0, max(x_freqs),0, max(y_freqs)),
                  origin='lower',
                  cmap='viridis')
        fig.colorbar(im, ax=ax[k+1], orientation='vertical')
        ax[k+1].set_title(f'2D window spectrum')
        ax[k].set_xlabel('X freqs')
        ax[k].set_ylabel('Y freqs')
